In [1]:
import momepy
import geopandas as gpd
import shapely
from shapely.geometry import LineString, Point
from shapely.ops import unary_union
import os
from dotenv import load_dotenv
import planetary_computer
import pystac_client
import dask.dataframe
import dgd
import dask.distributed
import deltalake
import shapely.geometry
import mercantile

/root/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [63]:
local_crs = 27700
place = "glasgow"
lnglat = (-4.251846930489373, 55.86421405612109)
dist = 30000
country = "united kingdom"

In [58]:
# Create a GeoDataFrame with the specified point
gdf = gpd.GeoDataFrame(geometry=[Point(lnglat[0], lnglat[1])], crs=4326)

# Reproject to a coordinate system that uses meters (UTM)
gdf_utm = gdf.to_crs(epsg=local_crs)

# Create a 50 km buffer around the point
buffer = gdf_utm.buffer(1000)  # 50 km buffer

# Reproject buffer back to original CRS
area_of_interest = buffer.to_crs(epsg=4326)

# Plottin
area_of_interest.explore()

area_of_interest = area_of_interest[0]

In [66]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("ms-buildings")

asset = collection.assets["delta"]

storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}
table = deltalake.DeltaTable(asset.href, storage_options=storage_options)

quadkeys = [
    int(mercantile.quadkey(tile))
    for tile in mercantile.tiles(*area_of_interest.bounds, zooms=9)
]
quadkeys

uris = table.file_uris([("quadkey", "in", quadkeys)])
uris

df = dgd.read_parquet(uris, storage_options=storage_options)
df


,geometry,meanHeight,RegionName,quadkey
npartitions=1,,,,
,geometry,float32,category[known],category[known]
,...,...,...,...


In [64]:
subset = dgd.sjoin(df, area_of_interest, how="inner", op="intersects").compute()
subset.explore()

ValueError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: empty urlpath sequence

In [49]:
asset = collection.assets["delta"]